In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import sys
from glob import glob
import math
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.misc import imread
from scipy.misc import imresize
import urllib.request
import datetime
import traceback
import redis
import face_recognition
from time import sleep

from config import load_config
from nnet import predict
from util import visualize
from dataset.pose_dataset import data_to_input
from PIL import Image
import cv2

/usr/local/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
FOREHEAD = 'forehead'
CHIN = 'chin'
SHOULDER = 'shoulder'
ELBOW = 'elbow'
WRIST = 'wrist'
HIP = 'hip'
KNEE = 'knee'
ANKLE = 'ankle'
POSITION_1_PCT_X_THRESHOLD = 0.3
POSITION_1_PCT_Y_THRESHOLD = 0.3
JOINT_MATCH_THRESHOLD = 0.9
CENTER_CHANGE_ALPHA = 0.8
# body_graph = {}
# body_graph[FOREHEAD] = [CHIN]
# body_graph[CHIN] = [FOREHEAD, SHOULDER]
# body_graph[SHOULDER] = [CHIN, ELBOW, HIP]
# body_graph[ELBOW] = [SHOULDER, WRIST]
# body_graph[WRIST] = [ELBOW]
# body_graph[HIP] = [SHOULDER, KNEE]
# body_graph[KNEE] = [HIP, ANKLE]
# body_graph[ANKLE] = [KNEE]
LEFT_RESTING_POSITION = {
    'name': 'Left resting position',
    'description': 'Left hand to the side of your hips resting loose.'
}
RIGHT_RESTING_POSITION = {
    'name': 'Right resting position',
    'description': 'Right hand to the side of your hips resting loose.'
}
LEFT_SIDE_POSITION = {
    'name': 'Left side position'
}
RIGHT_SIDE_POSITION = {
    'name': 'Right side position',
}


physiotherapy_routine1 = [
    {'commentary': 'Today, we will do right hand movements. This physiotherapy routine will help you get back the full functioning of your right arms.',
        'steps':[
    LEFT_RESTING_POSITION,
    LEFT_SIDE_POSITION,
]*2},{
        'commentary': 'You are doing good. Just 3 more!',
        'steps': [
            LEFT_RESTING_POSITION,
            LEFT_SIDE_POSITION
        ]*3
    }
]

physiotherapy_routine2 = [
    {'commentary': 'This routine will help you relieve your shoulder pain. Lets start with left hand movements.',
        'steps':[
    RIGHT_RESTING_POSITION,
    RIGHT_SIDE_POSITION,
]*2},{
        'commentary': 'You are doing good. Now to the right 3 times.',
        'steps': [
            LEFT_RESTING_POSITION,
            LEFT_SIDE_POSITION
        ]*3
    }
]
PROFILES = [
{
    'name': 'Person1',
    'image_path': 'data/known_images/name_of_image.jpg',
    'routine_spec': physiotherapy_routine1
}
]


In [4]:
def enrich_profile(profile):
    profile['image'] = face_recognition.load_image_file(profile['image_path'])
    profile['face_encoding'] = face_recognition.face_encodings(profile['image'])[0]
    return profile

In [5]:
ENRICHED_PROFILES = {}
for profile in PROFILES:
    ENRICHED_PROFILES[profile['name']] = enrich_profile(profile)

FileNotFoundError: [Errno 2] No such file or directory: 'data/known_images/name_of_image.jpg'

In [6]:
cfg = load_config("pose_cfg.yaml")
# Load and setup CNN part detector
sess, inputs, outputs = predict.setup_pose_prediction(cfg)

INFO:tensorflow:Restoring parameters from models/mpii/mpii-single-resnet-101


INFO:tensorflow:Restoring parameters from models/mpii/mpii-single-resnet-101


In [7]:
def read_from_stream(url):
    stream = urllib.request.urlopen(url)
    bytes1 = bytes()
    while True:
        bytes1 += stream.read(1024)
        a = bytes1.find(b'\xff\xd8')
        b = bytes1.find(b'\xff\xd9')
        if a != -1 and b != -1:
            jpg = bytes1[a:b+2]
            bytes1 = bytes1[b+2:]
            yield cv2.imdecode(np.fromstring(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)

In [8]:
all_joints = cfg.all_joints
all_joints_names = cfg.all_joints_names
all_joints_names
all_joints

[[0, 5], [1, 4], [2, 3], [6, 11], [7, 10], [8, 9], [12], [13]]

In [9]:
all_joints_names

['ankle', 'knee', 'hip', 'wrist', 'elbow', 'shoulder', 'chin', 'forehead']

In [10]:
def get_index(part, key, idx):
    try:
        return part[key][idx]
    except:
        return np.nan

In [11]:
def filter_confident_pose_parts(pose_dict):
    filtered_pose_dict = {}
    for joint_name, parts in pose_dict.items():
        filtered_pose_dict[joint_name] = [p for p in parts if p[-1] > JOINT_MATCH_THRESHOLD]
    return filtered_pose_dict

In [12]:
def detect_pose(image):
    image_batch = data_to_input(image)

    # Compute prediction with the CNN
    outputs_np = sess.run(outputs, feed_dict={inputs: image_batch})
    scmap, locref, _ = predict.extract_cnn_output(outputs_np, cfg)

    # Extract maximum scoring location from the heatmap, assume 1 person
    pose = predict.argmax_pose_predict(scmap, locref, cfg.stride)
    return pose


In [13]:
def detect_exercise1_pos1(lr_parts):
    dir_pos1_detected = {}
    for direction, parts in lr_parts.items():
        xs = np.array([get_index(parts, SHOULDER, 0), get_index(parts, WRIST, 0), get_index(parts, ELBOW, 0)])
        mean = xs.mean()

        # Within % of the mean
        decision_on_x = all(abs((xs - mean) / xs) < POSITION_1_PCT_X_THRESHOLD)
        
        decision_on_y = get_index(parts, SHOULDER, 1) < get_index(parts, WRIST, 1) and get_index(parts, ELBOW, 1) < get_index(parts, WRIST, 1)
        dir_pos1_detected[direction] = decision_on_x and decision_on_y
    return dir_pos1_detected

In [14]:
def detect_exercise1_pos2(lr_parts):
    dir_pos_detected = {}
    for direction, parts in lr_parts.items():
        ys = np.array([get_index(parts, SHOULDER, 1), get_index(parts, WRIST, 1), get_index(parts, ELBOW, 1)])
        mean = ys.mean()
        # Within % of the mean
        decision_on_y =  all(abs((ys - mean) / ys) < POSITION_1_PCT_Y_THRESHOLD)
        if direction == 'left':
            decision_on_x =  get_index(parts, SHOULDER, 0) > get_index(parts, WRIST, 0) and get_index(parts, ELBOW, 0) > get_index(parts, WRIST, 0)
        else:
            decision_on_x =  get_index(parts, SHOULDER, 0) < get_index(parts, WRIST, 0) and get_index(parts, ELBOW, 0) < get_index(parts, WRIST, 0)
        dir_pos_detected[direction] = decision_on_x and decision_on_y
    return dir_pos_detected

In [15]:
def pose_to_dict(pose):
    poses_dict = {}
    for i, joint_name in enumerate(all_joints_names):
        poses_dict[joint_name] = [pose[j] for j in all_joints[i]]
    return poses_dict

In [16]:
def get_left_right_parts(pose_dict, x_center):
    left_parts = {}
    right_parts = {}
    for joint_name, jointcoords in pose_dict.items():
        if joint_name != CHIN and joint_name != FOREHEAD:
            for j in jointcoords:
                if j[0] < x_center:
                    left_parts[joint_name] = j
                elif j[0] > x_center:
                    right_parts[joint_name] = j
    return {'left': left_parts, 'right': right_parts}

In [17]:
def find_center_x(pose_dict):
    """
        Finds the central point (x) to divide into left and right
    """
    shoulders = np.array(pose_dict[SHOULDER])
    matched_shoulders = shoulders[(shoulders[:,2] > JOINT_MATCH_THRESHOLD)]
    hips = np.array(pose_dict[HIP])
    matched_hips = hips[(hips[:,2] > JOINT_MATCH_THRESHOLD)]
    chin = pose_dict[CHIN][0]
    matched_chin = chin[chin[2] > JOINT_MATCH_THRESHOLD]
    forehead = pose_dict[FOREHEAD][0]
    matched_forehead = forehead[forehead[2] > JOINT_MATCH_THRESHOLD]


    mean_shoulders = np.nan if(len(matched_shoulders) < 2) else np.mean(matched_shoulders, axis=0)
    mean_shoulders

    mean_hips = np.nan if(len(matched_hips) < 2) else np.mean(matched_hips, axis=0)
    mean_hips

    matched_shoulders, matched_hips, matched_chin, matched_forehead

    mean_candidates = [mean_hips, mean_shoulders, matched_chin, matched_forehead]

    return np.mean(np.array(list(filter(lambda m: not np.all(np.isnan(m)), mean_candidates))), axis=0)[0][0]

In [18]:
def check_pose(frame, pose):
    pose_dict = pose_to_dict(pose)
    
    return pose_dict

In [19]:
def plot_pose(frame, pose, frame_size_multiplier):
    for i, joint_indices in enumerate(all_joints):
            for joint_index in joint_indices:
                (x, y, prob) = pose[joint_index]
                if prob <= 0.6:
                    continue
                x,y = int(x*frame_size_multiplier), int(y*frame_size_multiplier)
                joint_name = all_joints_names[i]
                #print(x, y, all_joints_names[i])
                cv2.circle(frame, (x, y), int(1.5*frame_size_multiplier), (255,0,0), 1)
                cv2.putText(frame, "{} - ({},{})".format(joint_name, x, y), (x + int(1.3*frame_size_multiplier), y + int(1.3*frame_size_multiplier)), cv2.FONT_HERSHEY_DUPLEX, 0.2*frame_size_multiplier, (255, 255, 255))

In [20]:
def plot_exercise_detects(frame, lr_parts):
    detectors = [
        {'fn': detect_exercise1_pos1, 'results': [
            {'expected': {'left': True}, 'message': 'Right Resting position', 'position': LEFT_RESTING_POSITION},
             {'expected': {'right': True}, 'message': 'Left Resting position', 'position': RIGHT_RESTING_POSITION}
        ]
        },
        {'fn': detect_exercise1_pos2, 'results': [
           {'expected': {'left': True}, 'message': 'Right side arms position', 'position': LEFT_SIDE_POSITION},
           {'expected': {'right': True}, 'message': 'Left side arms position', 'position': RIGHT_SIDE_POSITION} 
        ]}
    ]
    i_matched = 0
    curr_positions = []
    for detector in detectors:
        fn = detector['fn']
        res = fn(lr_parts)
        expected_results = detector['results']
        for expected_spec in expected_results:
            kv = expected_spec['expected']
            msg = expected_spec['message']
            if all(k in res and res[k] == v for k,v in kv.items()):
                i_matched+=1
                curr_positions.append(expected_spec['position'])
                cv2.putText(frame, "{}".format(msg), (30,i_matched*30), cv2.FONT_HERSHEY_DUPLEX, 1.2, (0, 0, 255))
    return curr_positions

In [21]:
class PhysiotherapyRoutine():
    def __init__(self, profile, redis_cli):
        self._profile = profile
        self.routine_spec = profile['routine_spec']
        self._set_number = 0
        self._set_step_position = 0
        self._remaining_steps = sum(len(r['steps']) for r in self.routine_spec) - 1
        self._frame_text = None
        self._is_completed = False
        self._last_activity_detected_time = datetime.datetime.now()
        self._redis_cli = redis_cli
        
    def start(self, intro_message):
        # Speak first discription
        print("Starting routine")
        self.speak(intro_message)
        sleep(0.1)
#         self.speak('Starting routine')
        self._personalize_speak(self.routine_spec[0]['commentary'])
        self._plot_text(self._remaining_steps, frame)
    
    def on_tick(self, frame):
        self._plot_text(self._frame_text, frame)
    
    def _personalize_speak(self, msg):
        try:
            self.speak("{}, {}".format(self._profile['name'], msg))
        except Exception as e:
            raise e
    def speak(self, msg):
        try:
            self._redis_cli.rpush('speak', msg)
        except:
            pass
        print("Speaking {}".format(msg))
    
    def handle_detected_positions(self, detected_positions, frame):
        expected_position_to_advance = self.routine_spec[self._set_number]['steps'][self._set_step_position]
        if any([d == expected_position_to_advance for d in detected_positions]):
            if self._is_detection_valid(detected_positions):
                self._advance_to_next(frame)
        
    def _is_detection_valid(self, detected_positions):
        # Check previous timstamp
        if self._is_completed:
            return False
        detected_time = datetime.datetime.now()
        if (detected_time - self._last_activity_detected_time).microseconds/1000 > 500:
            self._last_activity_detected_time = detected_time
            return True
        else:
            return False

    def _advance_to_next(self, frame):
        if self._has_next_step_in_set():
            self._set_step_position += 1
            self._remaining_steps -= 1
            self._plot_text("{}".format(self.get_remaining_steps()), frame)
        elif self._has_next_set():
            self._set_number += 1
            self._remaining_steps -= 1
            self._set_step_position = 0
            self._plot_text("{}".format(self.get_remaining_steps()), frame)
            # Commentary
            self._speak_commentary()
        else:
            # Nothing left
            self._is_completed = True
            self._personalize_speak('This concludes your session for today, see you tomorrow!')
    
    def _speak_commentary(self):
        self._personalize_speak(self.routine_spec[self._set_number]['commentary'])
    
    def _has_next_step_in_set(self):
        try:
            return self.routine_spec[self._set_number]['steps'][self._set_step_position+1] is not None
        except:
            return False
    def _has_next_set(self):
        try:
            return self.routine_spec[self._set_number+1]['steps'][0] is not None
        except:
            return False
        
    def _plot_text(self, text, frame):
        self._frame_text = str(text)
        cv2.putText(frame, str(text), (70,70), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255))

    def get_remaining_steps(self):
        return self._remaining_steps
#     def _has_next_step(self):
#         return self._has_next_step_in_set() or self._has_next_set()
    def is_completed(self):
        return self._is_completed
    
    def is_running(self):
        return not self._is_completed


In [22]:
def find_profile_in_frame(frame):
    try:
        face_locations = face_recognition.face_locations(frame)
        face_encodings = face_recognition.face_encodings(frame, face_locations)
        for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                #for name, profile in ENRICHED_PROFILES.items():
                profile = list(ENRICHED_PROFILES.values());
                encoding1 = profile[0]['face_encoding']
                match = face_recognition.compare_faces(encoding1, face_encoding)
                print(match)
                if match[0]:
                    return profile[0]
                else:
                    return None
    except Exception as e:
        # Silently eat the exception
        raise e
        return None

In [23]:
resize_ratio = 0.55 # To be adjusted on basis of input frame size and resolution
frame_size_multiplier = 1./resize_ratio

process_this_frame = True
frameNo = 1
prev_pose = None
center_x = None
redis_cli = redis.Redis('localhost')
routine = None
found_profile = None
completed_time = None
is_in_completion_period = False

while True:
    video_capture = cv2.VideoCapture(0)
    ret, frame = video_capture.read(0)
# for frame in read_from_stream('stream url'): ## If reading input from a different source and streaming via http server
    frameNo += 1
#     frame = cv2.flip( frame, -1 ) ## For inverting the frames
    if completed_time is not None:
        is_in_completion_period = time.time() - completed_time < 5
        if is_in_completion_period:
            continue
    
    small_frame = cv2.resize(frame, (0, 0), fx=resize_ratio, fy=resize_ratio)
    if routine is None or not routine.is_running():
        found_profile = find_profile_in_frame(small_frame)
        if found_profile:
            routine = PhysiotherapyRoutine(found_profile, redis_cli)
            # Start the session!
            routine.start(intro_message="Hi {}, it is great to see you after yesterdays session.".format(found_profile['name']))

    if routine is not None and routine.is_running() and frameNo%7 == 0:
        pose = detect_pose(small_frame)
        pose_dict = filter_confident_pose_parts(pose_to_dict(pose))
        prev_pose = pose
        try:
            curr_center = find_center_x(pose_dict)

            if center_x is not None:
                center_x = center_x*CENTER_CHANGE_ALPHA + (1-CENTER_CHANGE_ALPHA)*curr_center
            else:
                center_x = curr_center
        except:
            pass


    if routine is not None and routine.is_running() and prev_pose is not None:
        try:
            plot_pose(frame, prev_pose, frame_size_multiplier)
            lr_parts = get_left_right_parts(pose_dict, center_x)
            detected_positions = plot_exercise_detects(frame, lr_parts)
            routine.handle_detected_positions(detected_positions, frame)
        except Exception as e:
            print(e)
    if routine is not None and routine.is_running():
        routine.on_tick(frame)
    if routine is not None and routine.is_completed():
        routine = None
        is_in_completion_period = True
        completed_time = time.time()
        found_profile = None
    cv2.imshow('Video', frame) 

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Release handle to the webcam
video_capture.release()
#cv2.destroyAllWindows()    

IndexError: list index out of range

In [ ]:
video_capture.release()
#cv2.destroyAllWindows()